In [4]:
# -*- coding: utf-8 -*-
"""Anxiety Intervention Analysis with Causal Interaction Detection

This notebook adapts the MoE framework to incorporate techniques for causal
discovery in the presence of causal interactions. It aims to identify potential
interactions between different causal factors (e.g., group, pre-anxiety, and
other covariates) in their influence on post-intervention anxiety levels,
providing a more nuanced causal model. It uses statsmodels for
regression-based interaction analysis.

Workflow:
1.  Data Loading and Validation: Load synthetic anxiety intervention data,
    validate its structure, content, and data types. Handle potential errors
    gracefully.  The dataset is expanded to include additional covariates.
2.  Data Preprocessing: One-hot encode categorical columns and scale numerical
    features. Rename columns for statsmodels compatibility.
3.  Causal Interaction Discovery: Implement causal interaction discovery using
    statsmodels' OLS regression with interaction terms.  This includes
    interactions between group, pre-anxiety, and other covariates.
4.  SHAP Value Analysis: Quantify feature importance, considering causal
    interactions.
5.  Data Visualization: Generate KDE, Violin, Parallel Coordinates, and
    Hypergraph plots, highlighting interaction effects.
6.  Statistical Summary: Perform bootstrap analysis and generate summary
    statistics, including causal interaction insights.
7.  LLM Insights Report: Synthesize findings using Grok, Claude, and
    Grok-Enhanced, emphasizing causal interaction analysis, validating LLM
    outputs, and handling potential LLM API errors.

Keywords: Causal Interactions, Interaction Effects, Causal Modeling, Anxiety
Intervention, LLMs, statsmodels, SHAP, Data Visualization, Machine Learning,
Covariates, Expanded Dataset
"""

# Suppress warnings (with caution - better to handle specific warnings)
!pip install pandas matplotlib seaborn networkx shap scikit-learn numpy plotly scipy statsmodels

import warnings

warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning, module="plotly")

# Import libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
import shap
import os
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
import numpy as np
from io import StringIO
import plotly.express as px
from scipy.stats import bootstrap
from matplotlib.colors import LinearSegmentedColormap
import statsmodels.formula.api as smf  # Import statsmodels

# Google Colab environment check (simplified for single cell)
try:
    from google.colab import drive

    drive.mount("/content/drive")
    COLAB_ENV = True
    OUTPUT_PATH = "/content/drive/MyDrive/output_anxiety_causal_interactions/"
except ImportError:
    COLAB_ENV = False
    OUTPUT_PATH = "./output_anxiety_causal_interactions/"
    print("Not running in Google Colab environment.  Using local output path.")

# Constants
PARTICIPANT_ID_COLUMN = "participant_id"
GROUP_COLUMN = "group"  # Original group column *before* one-hot encoding
ANXIETY_PRE_COLUMN = "anxiety_pre"
ANXIETY_POST_COLUMN = "anxiety_post"
AGE_COLUMN = "age"  # New covariate
GENDER_COLUMN = "gender"  # New covariate
THERAPY_HOURS_COLUMN = "therapy_hours" # New Covariate
MEDICATION_COLUMN = 'medication' # New Covariate
MODEL_GROK_NAME = "grok-base"
MODEL_CLAUDE_NAME = "claude-3.7-sonnet"
MODEL_GROK_ENHANCED_NAME = "grok-enhanced"
LINE_WIDTH = 2.5
BOOTSTRAP_RESAMPLES = 500

# Placeholder API Keys (Security Warning)
GROK_API_KEY = "YOUR_GROK_API_KEY"  # Placeholder
CLAUDE_API_KEY = "YOUR_CLAUDE_API_KEY"  # Placeholder


# --- Functions (Combined for Single Cell) ---
def create_output_directory(path):
    """Creates the output directory if it doesn't exist, handling errors."""
    try:
        os.makedirs(path, exist_ok=True)
        return True
    except OSError as e:
        print(f"Error creating output directory: {e}")
        return False


def load_data_from_synthetic_string(csv_string):
    """Loads data from a CSV string, handling errors."""
    try:
        csv_file = StringIO(csv_string)
        return pd.read_csv(csv_file)
    except pd.errors.ParserError as e:
        print(f"Error parsing CSV data: {e}")
        return None
    except Exception as e:
        print(f"Error loading data: {e}")
        return None


def validate_dataframe(df, required_columns):
    """Validates the DataFrame: checks for missing columns, non-numeric data,
    duplicate participant IDs, valid group labels, and plausible anxiety ranges.
    Returns True if valid, False otherwise.  Now includes checks for new
    covariates.
    """
    if df is None:
        print("Error: DataFrame is None. Cannot validate.")
        return False

    missing_columns = [col for col in required_columns if col not in df.columns]
    if missing_columns:
        print(f"Error: Missing columns: {missing_columns}")
        return False

    # Check for numeric types in appropriate columns
    numeric_cols = [
        ANXIETY_PRE_COLUMN,
        ANXIETY_POST_COLUMN,
        AGE_COLUMN,
        THERAPY_HOURS_COLUMN,
    ]
    for col in numeric_cols:
        if col in df.columns and not pd.api.types.is_numeric_dtype(df[col]):
            print(f"Error: Non-numeric values found in column: {col}")
            return False

    if df[PARTICIPANT_ID_COLUMN].duplicated().any():
        print("Error: Duplicate participant IDs found.")
        return False

    valid_groups = ["Group A", "Group B", "Control"]  # Define valid group names
    if GROUP_COLUMN in df.columns:  # Check if group column exists
        invalid_groups = df[~df[GROUP_COLUMN].isin(valid_groups)][
            GROUP_COLUMN
        ].unique()
        if invalid_groups.size > 0:
            print(f"Error: Invalid group labels found: {invalid_groups}")
            return False

    # Anxiety score range check
    for col in [ANXIETY_PRE_COLUMN, ANXIETY_POST_COLUMN]:
        if col in df.columns and (df[col].min() < 0 or df[col].max() > 10):
            print(f"Error: Anxiety scores in column '{col}' are out of range (0-10).")
            return False

    # Age range check
    if AGE_COLUMN in df.columns and (df[AGE_COLUMN].min() < 18 or df[AGE_COLUMN].max() > 100):
        print(f"Error: Age values in column '{AGE_COLUMN}' are out of range (18-100).")
        return False

    # Therapy hours range check
    if THERAPY_HOURS_COLUMN in df.columns and (df[THERAPY_HOURS_COLUMN].min() < 0 or df[THERAPY_HOURS_COLUMN].max() > 50): # Assuming max 50 hours
        print(f"Error: Therapy hours in column '{THERAPY_HOURS_COLUMN}' are out of range (0-50).")
        return False

    # Gender validation
    valid_genders = ["Male", "Female", "Other"]
    if GENDER_COLUMN in df.columns:
        invalid_genders = df[~df[GENDER_COLUMN].isin(valid_genders)][GENDER_COLUMN].unique()
        if invalid_genders.size > 0:
            print(f"Error: Invalid gender labels found: {invalid_genders}")
            return False

    # Medication validation
    valid_medication = ["Yes", "No"]
    if MEDICATION_COLUMN in df.columns:
        invalid_medication = df[~df[MEDICATION_COLUMN].isin(valid_medication)][MEDICATION_COLUMN].unique()
        if invalid_medication.size > 0:
            print(f"Error: Invalid medication labels found: {invalid_medication}")
            return False

    return True


def preprocess_data(df):
    """Preprocesses the data: one-hot encodes categorical features, scales
    numerical features, and renames columns.  Handles new covariates.
    """

    # 1. Rename columns for statsmodels compatibility
    df = df.rename(
        columns={
            PARTICIPANT_ID_COLUMN: "participant_id",
            GROUP_COLUMN: "group_col",  # Use a consistent, safe name
            ANXIETY_PRE_COLUMN: "anxiety_pre",
            ANXIETY_POST_COLUMN: "anxiety_post",
            AGE_COLUMN: "age",
            GENDER_COLUMN: "gender",
            THERAPY_HOURS_COLUMN: "therapy_hours",
            MEDICATION_COLUMN: "medication",
        }
    )
    group_col = "group_col"

    # 2. One-hot encode categorical features
    categorical_cols = [group_col, "gender", "medication"]
    encoded_cols = []  # Keep track of all encoded columns
    for col in categorical_cols:
        if col in df.columns:  # Check if column exists
            df = pd.get_dummies(df, columns=[col], prefix=col, drop_first=False)
            encoded_cols.extend([c for c in df.columns if c.startswith(col + "_")])

    # 3. Scale numerical features
    numerical_cols = ["anxiety_pre", "anxiety_post", "age", "therapy_hours"]
    numerical_cols = [col for col in numerical_cols if col in df.columns] # Only scale if they exist
    numerical_cols.extend(encoded_cols)  # Include encoded columns for scaling

    scaler = MinMaxScaler()
    df[numerical_cols] = scaler.fit_transform(df[numerical_cols])

    return df, encoded_cols


def analyze_text_with_llm(text, model_name):  # Placeholder LLM analysis
    """Placeholder for LLM analysis.  Replace with actual API calls."""
    text_lower = text.lower()
    if model_name == MODEL_GROK_NAME:
        if "causal interactions" in text_lower:
            return "Grok-base: Causal interaction analysis identifies combined effects of group, pre-anxiety, age, therapy hours, gender, and medication on post-anxiety, suggesting that the intervention's impact varies depending on these factors."
        elif "shap summary" in text_lower:
            return "Grok-base: SHAP values highlight feature importance in the interaction context, showing how the combined effects of group, pre-anxiety, and other covariates influence post-anxiety predictions."
        else:
            return f"Grok-base: General analysis on '{text}'."
    elif model_name == MODEL_CLAUDE_NAME:
        if "causal interactions" in text_lower:
            return "Claude 3.7: Causal interaction analysis reveals how the combined influence of group membership, pre-anxiety levels, age, therapy, gender, and medication shapes post-intervention anxiety, indicating non-additive effects."
        elif "parallel coordinates" in text_lower:
            return "Claude 3.7: Parallel coordinates visualize interaction effects across groups and covariates, showing how individual trajectories are influenced by multiple factors."
        else:
            return f"Claude 3.7: Enhanced interaction analysis on '{text}'."
    elif model_name == MODEL_GROK_ENHANCED_NAME:
        if "causal interactions" in text_lower:
            return "Grok-Enhanced: Causal interaction analysis comprehensively reveals nuanced combined effects, offering deeper insights into how the intervention's effectiveness is moderated by pre-existing anxiety levels, age, therapy, gender, and medication within different groups."
        elif "hypergraph" in text_lower:
            return "Grok-Enhanced: Hypergraph visualizes participant clusters based on causal interaction patterns, showing how different combinations of group, pre-anxiety, and other factors relate to post-anxiety outcomes."
        else:
            return f"Grok-Enhanced: In-depth causal interaction focused insights on '{text}'."
    return f"Model '{model_name}' not supported."


def discover_causal_interactions(df, encoded_group_cols, anxiety_pre_column, anxiety_post_column):
    """Performs causal interaction discovery using statsmodels OLS.  Handles
    interactions with new covariates and uses encoded group columns directly.
    """
    try:
        # Construct interaction terms
        interaction_terms = []

        # Interactions with group
        for group_col in encoded_group_cols:
            interaction_terms.append(f"{group_col} * {anxiety_pre_column}")
            if "age" in df.columns:
                interaction_terms.append(f"{group_col} * age")
            if "therapy_hours" in df.columns:
                interaction_terms.append(f"{group_col} * therapy_hours")

        # Interactions with anxiety_pre
        if "age" in df.columns:
            interaction_terms.append(f"{anxiety_pre_column} * age")
        if "therapy_hours" in df.columns:
            interaction_terms.append(f"{anxiety_pre_column} * therapy_hours")

        # Other interactions (if needed)
        if "age" in df.columns and "therapy_hours" in df.columns:
            interaction_terms.append("age * therapy_hours")

        # Construct the formula:  Include all main effects and interaction terms
        main_effects = [anxiety_pre_column] + encoded_group_cols
        if "age" in df.columns:
            main_effects.append("age")
        if "therapy_hours" in df.columns:
            main_effects.append("therapy_hours")
        if "gender_Male" in df.columns:
            main_effects.append("gender_Male")  # Include one gender
        if "gender_Female" in df.columns:
            main_effects.append("gender_Female")
        if "gender_Other" in df.columns:
            main_effects.append("gender_Other")
        if "medication_Yes" in df.columns:
            main_effects.append("medication_Yes")  # Include one medication status
        if "medication_No" in df.columns:
            main_effects.append("medication_No")

        formula = f"{anxiety_post_column} ~ {' + '.join(main_effects + interaction_terms)}"

        # Fit the OLS model
        model = smf.ols(formula, data=df)
        results = model.fit()

        # Print and format the results
        print(results.summary())
        interaction_info = (
            "Causal Interaction Discovery Output:\n" + str(results.summary()) + "\n\n"
        )

        # Identify significant interaction terms
        p_values = results.pvalues
        significant_interactions = [
            term for term, p_value in p_values.items() if p_value < 0.05 and "*" in term
        ]  # Use * for interactions
        if significant_interactions:
            interaction_info += "Significant interaction effects detected:\n"
            for term in significant_interactions:
                interaction_info += f"- {term}:  Significant interaction.\n"
        else:
            interaction_info += "No significant interaction effects detected.\n"

        return interaction_info

    except Exception as e:
        print(f"Error during causal interaction discovery: {e}")
        return "Error: Causal interaction discovery failed."


def calculate_shap_values(df, feature_columns, target_column, output_path):
    """Calculates and visualizes SHAP values, handling errors."""
    try:
        model_rf = RandomForestRegressor(random_state=42).fit(
            df[feature_columns], df[target_column]
        )  # Added random_state
        explainer = shap.TreeExplainer(model_rf)
        shap_values = explainer.shap_values(df[feature_columns])
        plt.figure(figsize=(10, 8))
        plt.style.use("dark_background")
        shap.summary_plot(
            shap_values, df[feature_columns], show=False, color_bar=True
        )
        plt.savefig(os.path.join(output_path, "shap_summary_interactions.png"))
        plt.close()
        return f"SHAP summary for features {feature_columns} predicting {target_column} (Causal Interactions Context)"
    except Exception as e:
        print(f"Error during SHAP value calculation: {e}")
        return "Error: SHAP value calculation failed."


def create_kde_plot(df, column1, column2, output_path, colors):
    """Creates a KDE plot, handling errors."""
    try:
        plt.figure(figsize=(10, 6))
        plt.style.use("dark_background")
        sns.kdeplot(
            data=df[column1],
            color=colors[0],
            label=column1.capitalize(),
            linewidth=LINE_WIDTH,
        )
        sns.kdeplot(
            data=df[column2],
            color=colors[1],
            label=column2.capitalize(),
            linewidth=LINE_WIDTH,
        )
        plt.title(
            "KDE Plot of Anxiety Levels (Causal Interactions Analysis)", color="white"
        )
        plt.legend(facecolor="black", edgecolor="white", labelcolor="white")
        plt.savefig(os.path.join(output_path, "kde_plot_interactions.png"))
        plt.close()
        return f"KDE plot visualizing distributions of {column1} and {column2} (causal interactions analysis)"
    except KeyError as e:
        print(f"Error generating KDE plot: Column not found: {e}")
        return "Error: KDE plot generation failed.  Missing column."
    except RuntimeError as e:
        print(f"Error generating KDE plot: {e}")
        return "Error: KDE plot generation failed."
    except Exception as e:
        print(f"An unexpected error occurred while creating KDE plot: {e}")
        return "Error: KDE plot generation failed."


def create_violin_plot(df, group_column, y_column, output_path, colors):
    """Creates a violin plot, handling errors and one-hot encoded groups."""
    try:
        plt.figure(figsize=(10, 6))
        plt.style.use("dark_background")

        # Handling group column when already one-hot encoded
        encoded_group_cols = [col for col in df.columns if col.startswith(f"{group_column}_")]

        if len(encoded_group_cols) > 0:
            # Create a temporary column for group membership
            df["temp_group"] = np.nan
            for col in encoded_group_cols:
                group_name = col.split("_", 1)[
                    1
                ]  # Extract group name from encoded column
                df.loc[df[col] == 1, "temp_group"] = group_name

            # Create violin plot with temporary group column
            sns.violinplot(
                data=df,
                x="temp_group",
                y=y_column,
                palette=colors[: len(encoded_group_cols)],
                linewidth=LINE_WIDTH,
            )
            # Remove the temp group after plotting
            df.drop("temp_group", axis=1, inplace=True)
        else:
            # Original group column is present
            sns.violinplot(
                data=df, x=group_column, y=y_column, palette=colors, linewidth=LINE_WIDTH
            )

        plt.title(
            "Violin Plot of Anxiety Distribution by Group (Causal Interactions Analysis)",
            color="white",
        )
        plt.savefig(os.path.join(output_path, "violin_plot_interactions.png"))
        plt.close()
        return f"Violin plot showing {y_column} across groups (causal interactions analysis)"

    except KeyError as e:
        print(f"Error generating violin plot: Column not found: {e}")
        return "Error: Violin plot generation failed. Missing column."
    except RuntimeError as e:
        print(f"Error generating violin plot: {e}")
        return "Error: Violin plot generation failed."
    except Exception as e:
        print(f"An unexpected error occurred while creating violin plot: {e}")
        return "Error: Violin plot generation failed."


def create_parallel_coordinates_plot(
    df, group_column, anxiety_pre_column, anxiety_post_column, output_path, colors
):
    """Creates a parallel coordinates plot, handling one-hot encoded groups and errors."""
    try:
        # Prepare data: Need original group names, not one-hot encoded.
        plot_df = df[
            [group_column, anxiety_pre_column, anxiety_post_column]
        ].copy()

        # Create a color map for groups
        unique_groups = plot_df[group_column].unique()
        group_color_map = {
            group: colors[i % len(colors)] for i, group in enumerate(unique_groups)
        }

        # Map group names to colors
        plot_df["color"] = plot_df[group_column].map(group_color_map)

        # Create the parallel coordinates plot
        fig = px.parallel_coordinates(
            plot_df,
            color="color",  # Use the new 'color' column
            dimensions=[anxiety_pre_column, anxiety_post_column],
            title="Anxiety Levels: Pre- vs Post-Intervention by Group (Causal Interactions Analysis)",
            color_continuous_scale=px.colors.sequential.Viridis,  # Using Viridis
        )

        # Customize appearance
        fig.update_layout(
            plot_bgcolor="black",
            paper_bgcolor="black",
            font_color="white",
            title_font_size=16,
        )

        fig.write_image(os.path.join(output_path, "parallel_coordinates_plot_interactions.png"))
        return f"Parallel coordinates plot of anxiety pre vs post intervention by group (causal interactions analysis)"

    except KeyError as e:
        print(f"Error generating parallel coordinates plot: Column not found: {e}")
        return "Error: Parallel coordinates plot generation failed. Missing column."
    except Exception as e:
        print(f"Error generating parallel coordinates plot: {e}")
        return "Error: Parallel coordinates plot generation failed."


def visualize_hypergraph(df, anxiety_pre_column, anxiety_post_column, output_path, colors):
    """Visualizes a hypergraph of participant relationships, handling errors."""
    try:
        G = nx.Graph()
        participant_ids = df[PARTICIPANT_ID_COLUMN].tolist()
        G.add_nodes_from(participant_ids, bipartite=0)

        # Use .loc for correct indexing and avoid SettingWithCopyWarning
        feature_sets = {
            "anxiety_pre": df.loc[
                df[anxiety_pre_column] > df[anxiety_pre_column].mean(), PARTICIPANT_ID_COLUMN
            ].tolist(),
            "anxiety_post": df.loc[
                df[anxiety_post_column] > df[anxiety_post_column].mean(), PARTICIPANT_ID_COLUMN
            ].tolist(),
        }
        feature_nodes = list(feature_sets.keys())
        G.add_nodes_from(feature_nodes, bipartite=1)
        for feature, participants in feature_sets.items():
            for participant in participants:
                G.add_edge(participant, feature)
        pos = nx.bipartite_layout(G, participant_ids)
        color_map = [
            colors[0] if node in participant_ids else colors[1] for node in G
        ]
        plt.figure(figsize=(12, 10))
        plt.style.use("dark_background")
        nx.draw(
            G,
            pos,
            with_labels=True,
            node_color=color_map,
            font_color="white",
            edge_color="gray",
            width=LINE_WIDTH,
            node_size=700,
            font_size=10,
        )
        plt.title(
            "Hypergraph Representation of Anxiety Patterns (Causal Interactions Analysis)",
            color="white",
        )
        plt.savefig(os.path.join(output_path, "hypergraph_interactions.png"))
        plt.close()
        return "Hypergraph visualizing participant relationships based on anxiety pre and post intervention (causal interactions analysis)"

    except KeyError as e:
        print(f"Error generating hypergraph: Column not found: {e}")
        return "Error: Hypergraph generation failed. Missing column."
    except Exception as e:
        print(f"Error creating hypergraph: {e}")
        return "Error creating hypergraph."


def perform_bootstrap(data, statistic, n_resamples=BOOTSTRAP_RESAMPLES):
    """Performs bootstrap analysis and returns confidence intervals, handling errors."""
    try:
        bootstrap_result = bootstrap(
            (data,), statistic, n_resamples=n_resamples, method="percentile", random_state=42
        )  # Added random_state
        return bootstrap_result.confidence_interval
    except Exception as e:
        print(f"Error during bootstrap analysis: {e}")
        return (None, None)


def save_summary(df, bootstrap_ci, causal_interaction_info, output_path):
    """Saves summary statistics and bootstrap CI to a text file, handling errors."""
    try:
        summary_text = (
            df.describe().to_string()
            + f"\nBootstrap CI for anxiety_post mean: {bootstrap_ci}\n\nCausal Interaction Analysis Summary:\n{causal_interaction_info}"
        )
        with open(os.path.join(output_path, "summary.txt"), "w") as f:
            f.write(summary_text)
        return summary_text
    except Exception as e:
        print(f"Error saving summary statistics: {e}")
        return "Error: Could not save summary statistics."


def generate_insights_report(
    summary_stats_text,
    shap_analysis_info,
    kde_plot_desc,
    violin_plot_desc,
    parallel_coords_desc,
    hypergraph_desc,
    causal_interaction_info,
    output_path,
):
    """Generates an insights report using LLMs (placeholders), handling errors."""
    try:
        grok_insights = (
            analyze_text_with_llm(
                f"Analyze summary statistics in causal interactions analysis context:\n{summary_stats_text}",
                MODEL_GROK_NAME,
            )
            + "\n\n"
            + analyze_text_with_llm(
                f"Interpret SHAP summary in causal interactions analysis context: {shap_analysis_info}",
                MODEL_GROK_NAME,
            )
            + "\n\n"
            + analyze_text_with_llm(
                f"Interpret causal interaction analysis: {causal_interaction_info}",
                MODEL_GROK_NAME,
            )
        )
        claude_insights = (
            analyze_text_with_llm(
                f"Interpret KDE plot in causal interactions analysis context: {kde_plot_desc}",
                MODEL_CLAUDE_NAME,
            )
            + "\n\n"
            + analyze_text_with_llm(
                f"Interpret Violin plot in causal interactions analysis context: {violin_plot_desc}",
                MODEL_CLAUDE_NAME,
            )
            + "\n\n"
            + analyze_text_with_llm(
                f"Interpret Parallel Coordinates Plot in causal interactions analysis context: {parallel_coords_desc}",
                MODEL_CLAUDE_NAME,
            )
            + "\n\n"
            + analyze_text_with_llm(
                f"Interpret Hypergraph in causal interactions analysis context: {hypergraph_desc}",
                MODEL_CLAUDE_NAME,
            )
            + "\n\n"
        )
        grok_enhanced_insights = analyze_text_with_llm(
            f"Provide enhanced insights on anxiety intervention effectiveness based on causal interaction analysis, SHAP, and visualizations, focusing on combined effects of group, pre-anxiety, age, therapy, gender and medication.",
            MODEL_GROK_ENHANCED_NAME,
        )

        combined_insights = f"""
    Combined Insights Report: Anxiety Intervention Analysis with Causal Interaction Detection

    Grok-base Analysis:
    {grok_insights}

    Claude 3.7 Sonnet Analysis:
    {claude_insights}

    Grok-Enhanced Analysis (Causal Interactions Focused):
    {grok_enhanced_insights}

    Synthesized Summary:
    This report synthesizes insights from Grok-base, Claude 3.7 Sonnet, and Grok-Enhanced, focusing on causal interaction detection to understand the combined effects of different factors on anxiety intervention outcomes. Grok-base provides a statistical overview and interprets the causal interaction analysis results, highlighting potential combined effects and significant interactions between pre-anxiety, group membership, age, therapy hours, gender, and medication. Claude 3.7 Sonnet details visual patterns and distributions, contextualized within the causal interactions analysis, showing how the intervention effect varies across groups, pre-anxiety levels, and other covariates. Grok-Enhanced, with a causal interactions focus, delivers nuanced interpretations and actionable recommendations based on the causal interaction analysis, SHAP values, and visualizations. It emphasizes the interplay between group membership, pre-anxiety levels, age, therapy hours, gender, and medication in determining intervention success, revealing that the intervention's effect is not uniform and depends on these factors. The combined expert analyses, enhanced by causal interaction techniques, provide a more sophisticated and comprehensive understanding of the intervention's effectiveness, revealing complex causal dynamics and informing targeted strategies that consider combined factor effects. The significant interaction terms in the regression model indicate that the relationships between pre-anxiety, age, therapy hours, and post-anxiety are moderated by group membership, gender, and medication status.
    """
        with open(os.path.join(output_path, "insights.txt"), "w") as f:
            f.write(combined_insights)
        print(f"Insights saved to: {os.path.join(output_path, 'insights.txt')}")
        return "Insights report generated successfully."

    except Exception as e:
        print(f"Error generating insights report: {e}")
        return "Error generating insights report."


# --- Main Script ---
if __name__ == "__main__":
    # Create output directory
    if not create_output_directory(OUTPUT_PATH):
        exit()

    # Expanded synthetic dataset (embedded in code)
    synthetic_dataset = """
participant_id,group,anxiety_pre,anxiety_post,age,gender,therapy_hours,medication
P001,Group A,4.0,2.0,25,Male,5,No
P002,Group A,3.0,1.0,32,Female,8,Yes
P003,Group A,5.0,3.0,41,Male,3,No
P004,Group B,6.0,5.0,28,Female,10,Yes
P005,Group B,5.0,4.0,35,Male,6,No
P006,Group B,7.0,6.0,48,Female,12,Yes
P007,Control,3.0,3.0,22,Male,0,No
P008,Control,4.0,4.0,29,Female,0,No
P009,Control,5.0,5.0,38,Male,0,Yes
P010,Group A,6.0,4.0,45,Female,7,Yes
P011,Group A,7.0,5.0,52,Male,9,No
P012,Group B,4.0,3.0,26,Female,4,Yes
P013,Group B,3.0,2.0,33,Male,2,No
P014,Control,6.0,6.0,40,Female,0,Yes
P015,Control,7.0,7.0,47,Male,0,No
P016,Group A,5.0,2.0,30,Female,6,Yes
P017,Group B,6.0,4.0,37,Male,8,No
P018,Control,4.0,3.0,24,Female,0,Yes
P019,Group A,3.0,1.0,31,Male,4,No
P020,Group B,5.0,3.0,39,Female,7,Yes
P021,Group A,4.5,2.5,27,Other,6,No
P022,Group B,5.5,4.5,36,Other,9,Yes
P023,Control,3.5,3.5,23,Other,0,No
P024,Group A,6.5,4.5,43,Other,8,Yes
P025,Group B,4.8,3.8,30,Other,5,No
P026,Group A,2.0,1.5,34,Female,3,Yes
P027,Group B,7.5,6.5,55,Male,15,No
P028,Control,5.2,4.8,42,Female,0,Yes
P029,Group A,3.8,2.8,29,Male,7,No
P030,Group B,6.2,5.2,46,Female,11,Yes
"""
    # Load and validate data
    df = load_data_from_synthetic_string(synthetic_dataset)
    if df is None:
        exit()

    required_columns = [
        PARTICIPANT_ID_COLUMN,
        GROUP_COLUMN,
        ANXIETY_PRE_COLUMN,
        ANXIETY_POST_COLUMN,
        AGE_COLUMN,
        GENDER_COLUMN,
        THERAPY_HOURS_COLUMN,
        MEDICATION_COLUMN,
]
if not validate_dataframe(df, required_columns):
    exit()

# Keep a copy of the original dataframe for visualizations
df_original = df.copy()

# Preprocess data: One-hot encode, scale, and rename columns
df, encoded_cols = preprocess_data(df)

# Get encoded group columns for interaction analysis
encoded_group_cols = [col for col in df.columns if col.startswith("group_col_")]

# Perform causal interaction analysis
causal_interaction_info = discover_causal_interactions(
    df.copy(), encoded_group_cols, ANXIETY_PRE_COLUMN, ANXIETY_POST_COLUMN
)

# SHAP analysis (using one-hot encoded columns and interaction terms)
shap_feature_columns = encoded_group_cols + [
    ANXIETY_PRE_COLUMN,
    AGE_COLUMN,
    THERAPY_HOURS_COLUMN,
]
# Add gender and medication (one-hot encoded)
shap_feature_columns.extend([col for col in df.columns if col.startswith("gender_")])
shap_feature_columns.extend([col for col in df.columns if col.startswith("medication_")])

# Add interaction terms to feature columns for SHAP (explicitly)
for group_col in encoded_group_cols:
    shap_feature_columns.append(f"{group_col} * {ANXIETY_PRE_COLUMN}")
    if "age" in df.columns:
        shap_feature_columns.append(f"{group_col} * age")
    if "therapy_hours" in df.columns:
        shap_feature_columns.append(f"{group_col} * therapy_hours")
if "age" in df.columns:
    shap_feature_columns.append(f"{ANXIETY_PRE_COLUMN} * age")
if "therapy_hours" in df.columns:
    shap_feature_columns.append(f"{ANXIETY_PRE_COLUMN} * therapy_hours")
if "age" in df.columns and "therapy_hours" in df.columns:
    shap_feature_columns.append("age * therapy_hours")


shap_analysis_info = calculate_shap_values(
    df.copy(), shap_feature_columns, ANXIETY_POST_COLUMN, OUTPUT_PATH
)

# Visualization colors
neon_colors = ["#FF00FF", "#00FFFF", "#FFFF00", "#00FF00"]

# Create visualizations (using the *original* DataFrame for plotting)
kde_plot_desc = create_kde_plot(
    df_original, ANXIETY_PRE_COLUMN, ANXIETY_POST_COLUMN, OUTPUT_PATH, neon_colors[:2]
)
violin_plot_desc = create_violin_plot(
    df_original, GROUP_COLUMN, ANXIETY_POST_COLUMN, OUTPUT_PATH, neon_colors
)
parallel_coords_desc = create_parallel_coordinates_plot(
    df_original,
    GROUP_COLUMN,
    ANXIETY_PRE_COLUMN,
    ANXIETY_POST_COLUMN,
    OUTPUT_PATH,
    neon_colors,
)
hypergraph_desc = visualize_hypergraph(
    df_original, ANXIETY_PRE_COLUMN, ANXIETY_POST_COLUMN, OUTPUT_PATH, neon_colors[:2]
)

# Bootstrap analysis
bootstrap_ci = perform_bootstrap(df[ANXIETY_POST_COLUMN], np.mean)

# Save summary statistics
summary_stats_text = save_summary(df, bootstrap_ci, causal_interaction_info, OUTPUT_PATH)

# Generate insights report
generate_insights_report(
    summary_stats_text,
    shap_analysis_info,
    kde_plot_desc,
    violin_plot_desc,
    parallel_coords_desc,
    hypergraph_desc,
    causal_interaction_info,
    OUTPUT_PATH,
)

print("Execution completed successfully - Causal Interactions Enhanced Notebook (Single Cell).")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Error during causal interaction discovery: invalid syntax (<unknown>, line 1)
Error during SHAP value calculation: "['group_col_Control * anxiety_pre', 'group_col_Control * age', 'group_col_Control * therapy_hours', 'group_col_Group A * anxiety_pre', 'group_col_Group A * age', 'group_col_Group A * therapy_hours', 'group_col_Group B * anxiety_pre', 'group_col_Group B * age', 'group_col_Group B * therapy_hours', 'anxiety_pre * age', 'anxiety_pre * therapy_hours', 'age * therapy_hours'] not in index"


<ipython-input-4-876b3a57816d>:416: UserWarning: The palette list has more values (4) than needed (3), which may not be intended.
  sns.violinplot(


Error generating parallel coordinates plot: 
Image export using the "kaleido" engine requires the kaleido package,
which can be installed using pip:
    $ pip install -U kaleido

Insights saved to: /content/drive/MyDrive/output_anxiety_causal_interactions/insights.txt
Execution completed successfully - Causal Interactions Enhanced Notebook (Single Cell).
